In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Add as add
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from numpy import array

In [2]:
df = pd.read_excel('telugu.xlsx')

In [3]:
df.head()

,Unnamed: 0,చిత్రం,శీర్షిక
0,0,1000268201_693b08cb0e.jpg,పింక్ డ్రెస్‌లో ఉన్న పిల్లవాడు ప్రవేశ మార్గంల...
1,1,1000268201_693b08cb0e.jpg,ఒక చెక్క భవనంలోకి వెళ్తున్న అమ్మాయి.
2,2,1000268201_693b08cb0e.jpg,ఒక చెక్క ప్లేహౌస్‌లోకి ఎక్కుతున్న ఒక చిన్న అమ...
3,3,1000268201_693b08cb0e.jpg,ఒక చిన్న అమ్మాయి తన ప్లే హౌస్‌కి మెట్లు ఎక్కు...
4,4,1000268201_693b08cb0e.jpg,పింక్ డ్రెస్‌లో ఒక చిన్న అమ్మాయి చెక్క క్యాబి...


In [4]:
x = df.iloc[:,2]
x

0        పింక్ డ్రెస్‌లో ఉన్న పిల్లవాడు ప్రవేశ మార్గంల...
1                    ఒక చెక్క భవనంలోకి వెళ్తున్న అమ్మాయి.
2        ఒక చెక్క ప్లేహౌస్‌లోకి ఎక్కుతున్న ఒక చిన్న అమ...
3        ఒక చిన్న అమ్మాయి తన ప్లే హౌస్‌కి మెట్లు ఎక్కు...
4        పింక్ డ్రెస్‌లో ఒక చిన్న అమ్మాయి చెక్క క్యాబి...
                              ...                        
4994     పర్వతం ముందు నిలబడి ఉన్న వ్యక్తి ఆకుపచ్చ టోపీ...
4995     ఒక బ్రౌన్ చౌ మిక్స్ డాగ్ గజిబిజిగా ఉన్న యార్డ...
4996     బురదతో కూడిన పెరట్లో గోధుమ రంగు కుక్క నిలబడి ...
4997     ఒక గోధుమరంగు కుక్క ఒక ఆకుపచ్చ వ్యాన్ మరియు కొ...
4998    ఒక ఇంటి పెరట్లో ఒక ఆకుపచ్చ వ్యాన్ ముందు ఒక పెద...
Name:  శీర్షిక, Length: 4999, dtype: object

In [147]:
d = {}
d1 = {}
c = 1
t = ""
for i in x:
    t = t + i
k=t.replace("\n", "")
k=k.replace("\u200c", "")
text = k
d["start"] = c
c = c + 1
d['end'] = c
c = c + 1
for i in text.split() :
    if i not in d:
        d[i] = c
        d1[c] = i
        c = c + 1
d,d1

({'start': 1,
  'end': 2,
  'పింక్': 3,
  'డ్రెస్లో': 4,
  'ఉన్న': 5,
  'పిల్లవాడు': 6,
  'ప్రవేశ': 7,
  'మార్గంలో': 8,
  'మెట్ల': 9,
  'సెట్': 10,
  'పైకి': 11,
  'ఎక్కుతున్నాడు.': 12,
  'ఒక': 13,
  'చెక్క': 14,
  'భవనంలోకి': 15,
  'వెళ్తున్న': 16,
  'అమ్మాయి.': 17,
  'ప్లేహౌస్లోకి': 18,
  'ఎక్కుతున్న': 19,
  'చిన్న': 20,
  'అమ్మాయి': 21,
  'తన': 22,
  'ప్లే': 23,
  'హౌస్కి': 24,
  'మెట్లు': 25,
  'ఎక్కుతోంది.': 26,
  'క్యాబిన్లోకి': 27,
  'వెళుతోంది.': 28,
  'నల్ల': 29,
  'కుక్క': 30,
  'మరియు': 31,
  'మచ్చల': 32,
  'పోరాడుతున్నాయి': 33,
  'మూడు': 34,
  'రంగుల': 35,
  'రోడ్డుపై': 36,
  'ఒకదానితో': 37,
  'ఒకటి': 38,
  'ఆడుకుంటున్నాయి.': 39,
  'గోధుమ': 40,
  'రంగు': 41,
  'మచ్చలతో': 42,
  'తెల్ల': 43,
  'వీధిలో': 44,
  'ఒకరినొకరు': 45,
  'చూసుకుంటున్నాయి.': 46,
  'వివిధ': 47,
  'జాతులకు': 48,
  'చెందిన': 49,
  'రెండు': 50,
  'కుక్కలు': 51,
  'ఒకదానికొకటి': 52,
  'చూస్తున్నాయి.': 53,
  'పేవ్మెంట్పై': 54,
  'కదులుతున్నాయి.పెయింట్తో': 55,
  'కప్పబడిన': 56,
  'గిన్నెలో': 57,
  'చేతులతో': 5

In [6]:
len(d)

6204

In [7]:
x_train = []
r = 0
for i in x :
    t = [d["start"]]
    k=i.replace("\n", "")
    k=k.replace("\u200c", "")
    for j in k.split():
        try :
            t.append(d[j])
        except:
            continue
    t.append(d["end"])
    x_train.append(t)
x_train

[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2],
 [1, 13, 14, 15, 16, 17, 2],
 [1, 13, 14, 18, 19, 13, 20, 17, 2],
 [1, 13, 20, 21, 22, 23, 24, 25, 26, 2],
 [1, 3, 4, 13, 20, 21, 14, 27, 28, 2],
 [1, 13, 29, 30, 31, 13, 32, 30, 33, 2],
 [1, 13, 29, 30, 31, 34, 35, 30, 36, 37, 38, 39, 2],
 [1, 13, 29, 30, 31, 40, 41, 42, 5, 43, 30, 44, 45, 46, 2],
 [1, 47, 48, 49, 50, 51, 36, 52, 53, 2],
 [1, 54, 50, 51, 52, 2],
 [1, 56, 13, 20, 21, 13, 57, 22, 58, 59, 60, 61, 62, 63, 64, 2],
 [1, 13, 65, 66, 61, 62, 13, 20, 21, 63, 64, 2],
 [1, 67, 13, 20, 21, 61, 5, 68, 69, 62, 70, 71, 2],
 [1, 61, 66, 62, 72, 13, 21, 63, 64, 2],
 [1, 72, 73, 67, 66, 74, 75, 2],
 [1, 13, 76, 77, 78, 79, 80, 30, 80, 81, 82, 2],
 [1, 13, 43, 83, 84, 85, 86, 77, 78, 13, 76, 87, 2],
 [1, 13, 76, 73, 77, 78, 88, 80, 81, 13, 43, 31, 89, 30, 90, 2],
 [1, 91, 92, 76, 22, 93, 94, 77, 78, 87, 2],
 [1, 95, 96, 97, 30, 98, 99, 77, 78, 100, 2],
 [1, 101, 41, 102, 103, 76, 104, 105, 2],
 [1, 13, 76, 101, 102, 31, 106, 107, 2],
 [1, 108, 3

In [48]:
im = pd.read_json('image_vectors.json')
im.head(10)

,image_name,vector
0,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
1,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
2,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
3,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
4,1000268201_693b08cb0e.jpg,"[0.0, 12.0946464539, 2.6597044468, 0.0, 0.0, 0..."
5,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
6,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
7,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
8,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."
9,1001773457_577c3a7d70.jpg,"[0.0, 10.5379838943, 17.9251937866, 0.0, 0.0, ..."


In [49]:
x_im = im['vector']

In [50]:
len(x_im),len(x_train)

(4999, 4999)

In [51]:
c = 0
max_length = 100
vocaab_size = 6205
epoch = 100
number_pics_per_batch = 3
X1 = []
X2 = []
y = []
steps = len(text)//number_pics_per_batch
for seq in x_train:
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocaab_size)[0]
        X1.append(x_im[c])
        X2.append(in_seq)
        y.append(out_seq)
    c = c + 1

In [12]:
def data_generator(x_train):
    c = 0
    max_length = 100
    vocaab_size = 6205
    epoch = 100
    number_pics_per_batch = 3
    X1 = []
    X2 = []
    y = []
    n = 0
    steps = len(text)//number_pics_per_batch
    while 1 :
        n = n + 1
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocaab_size)[0]
            X1.append(x_im[c])
            X2.append(in_seq)
            y.append(out_seq)
        c = c + 1
    # yield the batch data
        if n==number_pics_per_batch:
            yield [[array(X1), array(X2)], array(y)]
            X1, X2, y = list(), list(), list()
            n=0
            c = 0

In [13]:
embedding_dim = 200
inputs1=Input(shape=(2048,))
f1=Dropout(0.5)(inputs1)
f2=Dense(256,activation='relu')(f1)
inputs2=Input(shape=(max_length,))
se1 = Embedding(vocaab_size, embedding_dim, mask_zero=True)(inputs2)
se2=Dropout(0.5)(se1)
se3=LSTM(256)(se2)
decoder1=add()([f2,se3])
decoder2=Dense(256,activation='relu')(decoder1)
d1=Dropout(0.5)(decoder2)

decoder3=Dense(256,activation='relu')(d1)
d2=Dropout(0.5)(decoder3)

outputs=Dense(vocaab_size,activation='softmax')(d2)
model=Model(inputs=[inputs1,inputs2],outputs=outputs)



In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 200)     1241000     ['input_2[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_1[0][0]']                
                                                                                              

In [15]:
adam=optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam)

C:\Users\akash\Anaconda3\envs\tf_gpu\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
cur_inp =[]
for i in range(len(x_train[0])-1):
    cur_inp.append(x_train[0][i])
    out = x_train[0][i+1]
    print(cur_inp , out)

[1] 3
[1, 3] 4
[1, 3, 4] 5
[1, 3, 4, 5] 6
[1, 3, 4, 5, 6] 7
[1, 3, 4, 5, 6, 7] 8
[1, 3, 4, 5, 6, 7, 8] 9
[1, 3, 4, 5, 6, 7, 8, 9] 10
[1, 3, 4, 5, 6, 7, 8, 9, 10] 11
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11] 12
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 2


In [84]:
x_train_seq =[]
x_train_img =[]
y_train_seq =[]
for j in range(len(x_train)):
    cur_inp =[]
    for i in range(len(x_train[j])-1):
        cur_inp.append(x_train[j][i])
        out = x_train[j][i+1]
        x_train_seq.append(cur_inp.copy())
        y_train_seq.append(out)
        x_train_img.append(x_im[j].copy())
    print(j,end='\r')

In [85]:
x_train_seq=pad_sequences(x_train_seq, maxlen=max_length)

In [86]:
y_train_seq=to_categorical(y_train_seq, num_classes=vocaab_size)

In [92]:
generator = data_generator(x_train)
model.fit([np.array(x_train_img),np.array(x_train_seq)],y_train_seq, epochs=25, batch_size=1024, verbose=1,workers=-1)

Epoch 1/25
45/45 [==============================] - 20s 452ms/step - loss: 6.6353
Epoch 2/25
45/45 [==============================] - 21s 466ms/step - loss: 6.6244
Epoch 3/25
45/45 [==============================] - 21s 469ms/step - loss: 6.6190
Epoch 4/25
45/45 [==============================] - 21s 465ms/step - loss: 6.6233
Epoch 5/25
45/45 [==============================] - 21s 464ms/step - loss: 6.6140
Epoch 6/25
45/45 [==============================] - 21s 461ms/step - loss: 6.6046
Epoch 7/25
45/45 [==============================] - 21s 461ms/step - loss: 6.5806
Epoch 8/25
45/45 [==============================] - 21s 458ms/step - loss: 6.5899
Epoch 9/25
45/45 [==============================] - 21s 460ms/step - loss: 6.5763
Epoch 10/25
45/45 [==============================] - 21s 465ms/step - loss: 6.5697
Epoch 11/25
45/45 [==============================] - 21s 461ms/step - loss: 6.5693
Epoch 12/25
45/45 [==============================] - 21s 461ms/step - loss: 6.5521
Epoch 13/25
4

KeyboardInterrupt: 

In [159]:
o=model.predict([np.expand_dims(x_train_img[0],axis=0),np.expand_dims(x_train_seq[0],axis=0)])

In [154]:
dic ={}
for i in range(len(o[0])):
    dic[i+1] = o[0][i]

In [155]:
dic={k: v for k, v in sorted(dic.items(), key=lambda item: item[1],reverse=True)}

In [158]:
d1[list(dic.keys())[0]],list(dic.keys())[0],np.argmax(y_train_seq[0])

('పింక్', 3, 3)

In [118]:
model.save("nlp_md.h5")

In [132]:
{k: v for k, v in sorted(dic.items(), key=lambda item: item[1],des=True)}

TypeError: 'des' is an invalid keyword argument for sort()

In [123]:
dic

{1: 1.0285092e-06,
 2: 1.4859258e-06,
 3: 1.6832383e-06,
 4: 1.8227798e-06,
 5: 1.8234805e-06,
 6: 1.9192557e-06,
 7: 1.9465815e-06,
 8: 1.9854892e-06,
 9: 2.0138675e-06,
 10: 2.1033118e-06,
 11: 2.15178e-06,
 12: 2.169631e-06,
 13: 2.1739577e-06,
 14: 2.2562033e-06,
 15: 2.2797356e-06,
 16: 2.3261496e-06,
 17: 2.3548448e-06,
 18: 2.3710566e-06,
 19: 2.3978048e-06,
 20: 2.436315e-06,
 21: 2.4410806e-06,
 22: 2.4836372e-06,
 23: 2.5275026e-06,
 24: 2.5561503e-06,
 25: 2.6889734e-06,
 26: 2.7261738e-06,
 27: 2.7878227e-06,
 28: 2.8282916e-06,
 29: 2.8578097e-06,
 30: 2.902811e-06,
 31: 2.926627e-06,
 32: 2.978518e-06,
 33: 3.0686238e-06,
 34: 3.07445e-06,
 35: 3.1437944e-06,
 36: 3.1647041e-06,
 37: 3.1898746e-06,
 38: 3.2062987e-06,
 39: 3.2230748e-06,
 40: 3.2522155e-06,
 41: 3.2651562e-06,
 42: 3.2794212e-06,
 43: 3.3794545e-06,
 44: 3.4046054e-06,
 45: 3.4247862e-06,
 46: 3.4752766e-06,
 47: 3.7217135e-06,
 48: 3.7501807e-06,
 49: 3.7614884e-06,
 50: 3.801562e-06,
 51: 3.8214744e-06,